In [3]:
import pandas as pd
import json

# 加载数据
df = pd.read_csv('/kaggle/input/molecule/photoswitches.csv')

# 你想用的四个波长字段名
fields = {
    'E isomer pi-pi* wavelength in nm': 'E isomer π–π*',
    'E isomer n-pi* wavelength in nm': 'E isomer n–π*',
    'Z isomer pi-pi* wavelength in nm': 'Z isomer π–π*',
    'Z isomer n-pi* wavelength in nm': 'Z isomer n–π*'
}

data = []

for _, row in df.iterrows():
    prompt_parts = []
    for col, name in fields.items():
        value = row.get(col)
        if pd.notna(value):  # 如果该字段不为空
            prompt_parts.append(f"{name} = {int(value)} nm")

    # 至少有一个 prompt 才保留该样本
    if prompt_parts and pd.notna(row['SMILES']):
        prompt = "; ".join(prompt_parts) + "."
        completion = row['SMILES']
        data.append({
            "prompt": prompt,
            "completion": completion
        })

# 保存为 JSONL 文件（供 fine-tuning 使用）
with open("photoswitch_full_prompt.jsonl", "w") as f:
    for item in data:
        f.write(json.dumps(item) + "\n")

print(f"共生成 {len(data)} 个训练样本。")


共生成 405 个训练样本。


In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForLanguageModeling
from datasets import load_dataset, Dataset
import json

model_id = "deepseek-ai/deepseek-r1-distill-qwen-1.5b"

tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    device_map="auto"
)

per_device_train_batch_size = 1
gradient_accumulation_steps = 16
fp16 = True


tokenizer_config.json:   0%|          | 0.00/3.07k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/679 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.55G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

In [7]:
# 读取 JSONL 数据（你之前已经处理好）
data = []
with open("/kaggle/input/4wavelength/photoswitch_full_prompt.jsonl", "r") as f:
    for line in f:
        item = json.loads(line)
        prompt = item["prompt"]
        completion = item["completion"]
        full_text = prompt + " " + completion
        data.append({"text": full_text})

# 转换为 Hugging Face Dataset
dataset = Dataset.from_list(data)


In [ ]:
def tokenize_function(example):
    return tokenizer(example["text"], truncation=True, padding="max_length", max_length=128)

tokenized_dataset = dataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/405 [00:00<?, ? examples/s]

In [9]:
print(tokenizer.tokenize("CC1=CC=CC=C1N=N"))


['CC', '1', '=', 'CC', '=', 'CC', '=C', '1', 'N', '=N']


In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="no",
    learning_rate=0.01,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    num_train_epochs=3,
    weight_decay=0.01,
    fp16=True,
    logging_steps=10,
    save_total_limit=1,
    save_strategy="epoch"
)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()

trainer.save_model("/kaggle/working/fine_tuned_deepseek_photoswitch")

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-10-5dc2d6a84cc9>:17: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>